In [13]:
# This script is used for testing all adaptations and changes made

In [28]:
import pandas as pd
import numpy as np

from scipy import linalg
from scipy import stats

In [29]:
# Load data for change detection as a pandas data frame.
infys_df =  pd.read_csv("../data/iMAD/iMAD_Data_13_TreesOnly.csv") #need to change the dataset 
infys_df.head()

,Unnamed: 0,Cycle,Cluster_ID,AvgDbh,AvgCrownDiameter,AvgCrownHeight,AvgCrownArea,SpeciesCount,AvgTreeHeight,TreeCount,J,X,Y
0,1,1,77652,16.103571,3.937736,3.367925,15.409993,8,8.555357,56,0.440017,-92.433389,15.588935
1,2,1,28451,10.380952,2.209524,0.000000,3.956920,2,2.909524,21,0.334649,-99.546778,26.072916
2,3,1,55242,14.996429,4.439683,4.557937,17.199388,5,5.458333,84,0.505430,-104.218713,20.403546
3,4,1,20795,11.600000,3.169565,4.286957,8.243627,5,4.493548,31,0.770242,-108.957083,27.390907
4,5,1,53236,12.194444,4.886047,5.753488,20.022403,6,7.605556,54,0.841231,-98.775463,20.791278


In [30]:
# Create new ids by concatenating 'Conglomerado' and 'Sitio'.
    ## don't think I need to do that
    ## Use existing ID of Metadataset called Cluster_ID -> should be able to leave that one out


# As we want to compare cycle 1 vs cycle 2 lets separate them.   
## Stays the same for long dataformat
## might figure out how to 
infys_df_c1 = infys_df[infys_df['Cycle'] == 1]
infys_df_c2 = infys_df[infys_df['Cycle'] == 3]

In [31]:
# Find intersection of new ID variable: 'cng_sit', so obs. that are present in both cycles.
s1 = infys_df_c1['Cluster_ID']
s2 = infys_df_c2['Cluster_ID']
cluster_id_inter = pd.Series(list(set(s1).intersection(set(s2))))
## might have already done this in R by using my filter(muestreado1 == 1 & muestreado2 == 1)



In [32]:
# Filter out obs. that dont appear in cng_sit_inter.
infys_df_c1 = infys_df_c1[infys_df_c1['Cluster_ID'].isin(cluster_id_inter)]
infys_df_c2 = infys_df_c2[infys_df_c2['Cluster_ID'].isin(cluster_id_inter)]

In [33]:
# Select only variables to be utilized in change detection.
vars = ["SpeciesCount", "TreeCount", "J", "AvgTreeHeight", "AvgDbh", "AvgCrownDiameter", "AvgCrownHeight", "AvgCrownArea"]
nvars = len(vars)
infys_df_c1_c = infys_df_c1[infys_df_c1.columns.intersection(vars)]
infys_df_c2_c = infys_df_c2[infys_df_c2.columns.intersection(vars)]

c1_c_nans = infys_df_c1_c.isna().any(axis=1).to_numpy()
c2_c_nans = infys_df_c2_c.isna().any(axis=1).to_numpy()

In [34]:
# Create change data matrix, missing values mask and initial change weights.
dm = np.zeros((2 * nvars, infys_df_c1_c.shape[0]))
dm[0:nvars] = np.transpose(infys_df_c1_c.to_numpy())
dm[nvars:] = np.transpose(infys_df_c2_c.to_numpy())

nodataidx = c1_c_nans | c2_c_nans
gooddataidx = nodataidx == False
dm = dm[:, gooddataidx]
ngood = np.sum(gooddataidx)

In [36]:
# Change detection iterations.

# iteration of MAD   
wt = np.ones(int(ngood))
delta = 1.0
oldrho = np.zeros(nvars)
iter = 0
 
while (delta > 0.01) and (iter < 33): 
    print(iter)
    # Weighted covariance matrices and means.
    sumw = np.sum(wt)
    means = np.average(dm,axis=1, weights=wt)
    dmc = dm - means[:,np.newaxis]
    dmc = np.multiply(dmc,np.sqrt(wt))
    sigma = np.dot(dmc,dmc.T)/sumw
   
    s11 = sigma[0:nvars, 0:nvars]
    s22 = sigma[nvars:, nvars:]
    s12 = sigma[0:nvars, nvars:]
    s21 = sigma[nvars:, 0:nvars]
    
    # Solution of generalized eigenproblems.
    aux_1 = linalg.solve(s22, s21)
    lama, a = linalg.eig(np.dot(s12, aux_1), s11)
    aux_2 = linalg.solve(s11, s12)
    lamb, b = linalg.eig(np.dot(s21, aux_2), s22)
    
    
    # Sort a.  
    idx = np.argsort(lama)
    a = a[:, idx]
    
    # Sort b.        
    idx = np.argsort(lamb)
    b = b[:, idx]
    
    # Canonical correlations.        
    rho = np.sqrt(np.real(lamb[idx]))
    
    
    # Normalize dispersions.  
    tmp1 = np.dot(np.dot(a.T,s11), a)
    tmp2 = 1. / np.sqrt(np.diag(tmp1))
    tmp3 = np.tile(tmp2, (nvars, 1))
    a = np.multiply(a, tmp3)
    b = np.mat(b)
    tmp1 = np.dot(np.dot(b.T,s22), b)
    tmp2 = 1. / np.sqrt(np.diag(tmp1))
    tmp3 = np.tile(tmp2, (nvars, 1))
    b = np.multiply(b, tmp3)
        
    # Assure positive correlation
    tmp = np.diag(np.dot(np.dot(a.T,s12), b))
    b = np.dot(b,np.diag(tmp / np.abs(tmp)))

    # Canonical and MAD variates
    U = np.dot(a.T , (dm[0:nvars, :] - means[0:nvars, np.newaxis]))    
    V = np.dot(b.T , (dm[nvars:, :] - means[nvars:, np.newaxis]))          
    MAD = U - V  
    
            
    # New weights.        
    var_mad = np.tile(np.mat(2 * (1 - rho)).T, (1, ngood))    
    chisqr = np.sum(np.multiply(MAD, MAD) / var_mad, 0)
    wt = np.squeeze(1 - np.array(stats.chi2._cdf(chisqr, nvars)))
    
        
    # Continue iteration.        
    delta = np.sum(np.abs(rho - oldrho))
    oldrho = rho
    iter += 1
    
# reshape to original image size, by including nodata pixels   
# actually reshaping to image size plus one to add chi-squared values
MADout = np.zeros((int(nvars+1), infys_df_c1_c.shape[0]))
MADout[0:nvars, gooddataidx] = MAD
MADout[nvars, gooddataidx] = chisqr

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [37]:
# check whether adding chi-squared worked -> should be 9 rows (or in pyhton count: 0-8)
MADout_df = pd.DataFrame(MADout)
MADout_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,8522,8523,8524,8525,8526,8527,8528,8529,8530,8531
0,-6.853842e+00,-3.519086e+00,-9.705645e-02,4.895934e+00,-2.041761e+00,-2.196718e+00,0.0,1.757639e+00,-2.289390e+00,6.792918e-01,...,-1.039411e+01,5.831103e+00,-8.874683e-01,-6.773673e+00,2.149456e+00,-1.180171e+00,-3.729438e+00,-3.459147e-01,-4.858157e+00,-7.629861e+00
1,4.820803e+00,7.813935e-01,1.331455e+00,-3.128524e-01,8.361135e-01,-2.719518e-02,0.0,7.951475e-02,-5.619160e-01,-3.694988e-01,...,2.351077e+00,-1.765769e+00,2.293238e+00,1.696391e+00,-1.012566e+00,9.890892e-01,-2.893168e+00,2.636397e-01,2.143147e+00,6.254960e-01
2,1.881412e-01,2.217609e-01,3.621881e-01,-7.976074e-01,1.488390e+00,2.148146e+00,0.0,-5.129704e-02,8.477846e-02,1.198620e-01,...,1.168121e+00,7.925038e-02,1.369314e+00,6.594701e-01,4.205511e-02,2.126176e+00,3.129789e+00,-2.411566e+00,9.859682e-01,1.693667e+00
3,1.824207e+00,-7.666559e-02,-7.255799e-01,4.432983e+00,2.145143e-01,-2.138818e+00,0.0,8.184765e-01,-1.870737e+00,7.210314e-01,...,-1.760172e+00,-7.469340e-02,6.552507e-01,-1.012079e+00,-8.616765e-01,2.326294e+00,-4.080121e+00,-1.180767e+00,1.181224e+00,-3.749293e+00
4,3.169813e-01,-1.313266e+00,-4.128861e-01,1.716095e-01,-6.072130e-01,-2.546069e+00,0.0,5.302199e-01,-1.429823e+00,-8.225616e-01,...,-2.872359e+00,8.411540e-03,9.824917e-01,-2.717338e+00,-2.328813e-01,-1.803518e+00,-2.647364e+00,1.444700e+00,-1.618167e-01,-5.702842e-01
5,9.075044e-01,2.463664e+00,-1.981128e+00,2.963786e+00,-5.532205e-01,-1.190521e+00,0.0,-1.223466e+00,-1.035882e+00,7.695990e-01,...,1.695506e-01,4.205396e-01,-3.166186e+00,-9.349770e-01,-2.065366e+00,6.930676e-01,-3.183889e+00,-1.625003e-01,-1.008531e+00,-3.206430e+00
6,-1.995270e-01,-1.019715e+00,-5.705727e-01,8.968777e-01,-1.585134e+00,-2.831503e+00,0.0,2.413656e-01,-1.430415e+00,-7.258405e-02,...,-2.852549e+00,4.327424e-01,-5.594806e-01,-2.300282e+00,-1.326396e-01,-2.508202e+00,-2.282520e+00,1.198250e+00,-9.743297e-01,-1.671780e+00
7,1.300037e+00,9.583952e-01,-5.578135e-01,-2.878669e-01,-3.008908e-01,3.408851e-01,0.0,-8.016873e-01,5.078475e-02,-8.643210e-02,...,1.634693e+00,-1.230617e+00,-4.377003e-01,7.890085e-01,-1.015036e+00,4.623720e-01,5.690930e-01,8.327873e-02,6.160823e-01,6.939777e-02
8,4.568045e+14,4.421063e+15,2.299124e+15,5.126988e+15,2.513066e+15,9.646776e+15,0.0,8.692208e+14,2.921477e+15,4.802903e+14,...,9.501145e+15,2.412551e+14,5.552026e+15,7.152998e+15,2.153946e+15,6.766306e+15,1.161856e+16,1.859322e+15,1.366987e+15,7.748191e+15


In [38]:
# increase dimension of transposed dataframe by adding on column called chi_squared
vars_with_chi_squared = vars + ["chi_squared"]

# transpose matrix 
MADout = np.transpose(MADout).copy()
MADout_df = pd.DataFrame(MADout, columns=vars_with_chi_squared)
# should now have 9 columns and the last one should be chi_squared
MADout_df.head(10)

,SpeciesCount,TreeCount,J,AvgTreeHeight,AvgDbh,AvgCrownDiameter,AvgCrownHeight,AvgCrownArea,chi_squared
0,-6.853842,4.820803,0.188141,1.824207,0.316981,0.907504,-0.199527,1.300037,4.568045e+14
1,-3.519086,0.781394,0.221761,-0.076666,-1.313266,2.463664,-1.019715,0.958395,4.421063e+15
2,-0.097056,1.331455,0.362188,-0.725580,-0.412886,-1.981128,-0.570573,-0.557814,2.299124e+15
3,4.895934,-0.312852,-0.797607,4.432983,0.171610,2.963786,0.896878,-0.287867,5.126988e+15
4,-2.041761,0.836114,1.488390,0.214514,-0.607213,-0.553220,-1.585134,-0.300891,2.513066e+15
5,-2.196718,-0.027195,2.148146,-2.138818,-2.546069,-1.190521,-2.831503,0.340885,9.646776e+15
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
7,1.757639,0.079515,-0.051297,0.818477,0.530220,-1.223466,0.241366,-0.801687,8.692208e+14
8,-2.289390,-0.561916,0.084778,-1.870737,-1.429823,-1.035882,-1.430415,0.050785,2.921477e+15
9,0.679292,-0.369499,0.119862,0.721031,-0.822562,0.769599,-0.072584,-0.086432,4.802903e+14


In [39]:
# add coordinates
MADout_df["X"] = infys_df_c1["X"]
MADout_df["Y"] = infys_df_c1["Y"]
MADout_df.head(10)

,SpeciesCount,TreeCount,J,AvgTreeHeight,AvgDbh,AvgCrownDiameter,AvgCrownHeight,AvgCrownArea,chi_squared,X,Y
0,-6.853842,4.820803,0.188141,1.824207,0.316981,0.907504,-0.199527,1.300037,4.568045e+14,-92.433389,15.588935
1,-3.519086,0.781394,0.221761,-0.076666,-1.313266,2.463664,-1.019715,0.958395,4.421063e+15,-99.546778,26.072916
2,-0.097056,1.331455,0.362188,-0.725580,-0.412886,-1.981128,-0.570573,-0.557814,2.299124e+15,-104.218713,20.403546
3,4.895934,-0.312852,-0.797607,4.432983,0.171610,2.963786,0.896878,-0.287867,5.126988e+15,-108.957083,27.390907
4,-2.041761,0.836114,1.488390,0.214514,-0.607213,-0.553220,-1.585134,-0.300891,2.513066e+15,-98.775463,20.791278
5,-2.196718,-0.027195,2.148146,-2.138818,-2.546069,-1.190521,-2.831503,0.340885,9.646776e+15,-103.579314,23.316000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,-101.453185,26.362889
7,1.757639,0.079515,-0.051297,0.818477,0.530220,-1.223466,0.241366,-0.801687,8.692208e+14,-108.212148,28.779231
8,-2.289390,-0.561916,0.084778,-1.870737,-1.429823,-1.035882,-1.430415,0.050785,2.921477e+15,-103.285564,23.454722
9,0.679292,-0.369499,0.119862,0.721031,-0.822562,0.769599,-0.072584,-0.086432,4.802903e+14,-98.962972,16.723870


In [40]:
# create .csv
MADout_df.to_csv('iMAD_results_13_TreesOnly.csv', index=False)

In [ ]:
#### ENDE ####

In [54]:
## Beta

MADout_df["Y"] = infys_df_c1["std_chg"]
np.std(madout(lstone))
madout(lastone <2*np.std)=2


# Add chi squared variable to table

CHIsqr_df = pd.DataFrame(chisqr)


chisqr = np.transpose(chisqr).copy()
CHI_df = pd.DataFrame(chisqr)

MADout_df['CHI'] = CHI_df



# Assuming chisqr is a 1D numpy array with length equal to the number of rows in MADout
# We ensure chisqr is reshaped as a 2D column vector if it isn't already
chisqr_column = chisqr.reshape(-1, 1)

# Append chisqr as a new column to MADout
MADout = np.concatenate((MADout, chisqr_column), axis=1)




MADout_df.head()



SyntaxError: cannot assign to function call here. Maybe you meant '==' instead of '='? (54363561.py, line 5)